In [1]:
import math, os, sys
import json
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.offline as py
from pathlib import Path
from datetime import datetime
from ipywidgets import interactive, HBox, VBox
from ipywidgets import widgets
#py.init_notebook_mode()

sys.path.append('/home/eddyod/programming/activebrainatlas')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "activebrainatlas.settings")
import django
django.setup()

In [2]:
from neuroglancer.models import UrlModel, LayerData, CenterOfMass, Structure
from brain.models import Animal, ScanRun

In [11]:
pk = 211 # DK54
urlModel = UrlModel.objects.get(pk=pk)
animal = urlModel.animal
scanRun = ScanRun.objects.get(prep_id__exact=animal)
df = urlModel.points
df = df[(df.Layer == 'PM nucleus') | (df.Layer == 'premotor')]
sections = df['Section'].sort_values().unique().tolist()
section = df['Section'].min()
img_width = scanRun.width
img_height = scanRun.height
print('section is ', section, img_width, img_height)
df = df.loc[ df['Section'] == section]
section = str(section).zfill(3)
source = f"https://activebrainatlas.ucsd.edu/data/{animal}/www/{section}.png"

fig = go.FigureWidget([go.Scatter(y = df['Y'], x = df['X'], mode = 'markers')])
fig.add_layout_image(
        dict(
        x=0,
        sizex=img_width,
        y=-10000,
        sizey=img_height,
        xref="x",
        yref="y",
        yanchor="top",
        opacity=0.85,
        layer="below",
        name="showme",
        source=source,
        sizing="contain")
)
fig.update_layout(title=urlModel.comments)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_xaxes(range=[0, img_width])
fig.update_yaxes(range=[0, img_height],scaleanchor="x")
fig.update_layout(template="plotly_white")
fig['layout']['yaxis']['autorange'] = "reversed"

scatter = fig.data[0]

scatter.marker.opacity = 1
scatter.marker.size = 6
scatter.marker.color = 'yellow'

section_selector = widgets.Dropdown(
    options=list(df['Section'].sort_values().unique()),
    value=df['Section'].iloc[0],
    description='Section:',
)

def response(change):
    filter_list = [s for s in df['Section'] == section_selector.value]
    tmp_df = df[filter_list]
        
    x = tmp_df['X']
    y = tmp_df['Y']
    with fig.batch_update():
        fig.layout.xaxis.title = "X"
        fig.layout.yaxis.title = "Y"
        section = str(section_selector.value).zfill(3)
        fig.layout.images[0].source=f"https://activebrainatlas.ucsd.edu/data/{animal}/www/{section_selector.value}.png"
        scatter.x = x
        scatter.y = y
    with t.batch_update():
        t.layout.title = f"{len(tmp_df)} points on section {section_selector.value}"
        t.data[0].cells.values = [tmp_df[col] for col in tablecols]

section_selector.observe(response, names="value")

tablecols = ['Layer', 'X', 'Y', 'Section']
t = go.FigureWidget([go.Table(
    header=dict(values=tablecols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[df[col] for col in tablecols],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])

def selection_fn(trace,points,selector):
    t.layout.title = f"You have selected {len(points.point_inds)} points"
    t.data[0].cells.values = [df.loc[points.point_inds][col] for col in tablecols]

scatter.on_selection(selection_fn)

figure_container = widgets.VBox([fig, t])
widgets.VBox([section_selector, figure_container])

section is  98 60000 38000


    'd…

In [ ]:
#df2 = pd.read_csv('https://raw.githubusercontent.com/jonmmease/plotly_ipywidget_notebooks/master/notebooks/data/cars/cars.csv')
#df2.select_dtypes('int64').columns
sections = df['Section'].sort_values().unique().tolist()

In [33]:
pk = 211 # DK54
#pk = 200 # DK52
urlModel = UrlModel.objects.get(pk=pk)
animal = urlModel.animal
scanRun = ScanRun.objects.get(prep_id__exact=animal)
df = urlModel.points
df = df[(df.Layer == 'PM nucleus') | (df.Layer == 'premotor')]
sections = df['Section'].sort_values().unique().tolist()
#section = 272
section = df['Section'].min()
img_width = scanRun.width
img_height = scanRun.height

print('section is ', section, img_width, img_height)
print(df['Layer'].unique())
df.describe()

section is  98 60000 38000
['premotor']


,X,Y,Section
count,17.000000,17.000000,17.000000
mean,1040.529412,579.117647,106.941176
std,182.073858,78.316571,4.588989
min,815.000000,448.000000,98.000000
25%,839.000000,567.000000,106.000000
50%,1122.000000,613.000000,110.000000
75%,1193.000000,634.000000,110.000000
max,1261.000000,664.000000,114.000000


In [40]:
{k:'#{0:06X}'.format(k) for k in sections}

{98: '#000062', 102: '#000066', 106: '#00006A', 110: '#00006E', 114: '#000072'}